### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42630877,79085,9514,51530,0,0,1135,5,2025-04-10 22:00,2025-04-10 22:00,0,5,2645,4,1
34,42630478,34132,79085,3,0,0,1135,6,2025-04-11 18:22,2025-04-11 18:22,1,1,5894,4,1
35,42630878,9514,79085,51530,0,0,1136,0,2025-04-12 22:00,2025-04-12 22:00,0,1,5667,3,1
36,42410307,10520,79085,1349,7,70,1136,1,2025-04-13 21:59,2025-04-13 21:59,3,0,27392,5,1
37,42634638,79536,79085,51530,0,0,1136,2,2025-04-14 19:00,2025-04-14 19:00,2,0,2331,3,1
38,42635838,79085,15699,51530,0,0,1136,3,2025-04-15 22:00,2025-04-15 22:00,3,2,1778,4,1
39,42410319,79085,78671,1349,8,70,1136,4,2025-04-16 21:59,2025-04-16 21:59,0,1,11762,4,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,79085,47479,32407,3,2,3,0,0,3-4-3BC,8,9,10,42630877
1,9514,22474,37546,3,10,4,0,0,Concha Campions,9,9,9,42630877
0,34132,36137,35482,2,8,0,0,0,passo dobble,8,8,8,42630478
1,79085,33821,34476,1,7,4,1,0,4-2-4a,6,8,9,42630478
0,9514,20386,41078,1,8,3,0,0,Concha Campions,9,9,10,42630878
1,79085,49695,29003,6,2,0,0,0,4-2-4a,9,9,10,42630878
0,10520,19377,38266,1,17,1,0,0,_43111_,10,11,11,42410307
1,79085,50899,32010,0,2,4,2,0,5-4-1a,8,9,11,42410307
0,79536,35173,35278,1,7,3,0,0,c2,9,8,8,42634638
1,79085,35169,35064,1,9,2,1,0,4-3-3a,9,9,10,42634638


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1686

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,23775,8,24,8,0,0,26,6,24520026008001,1349
1,10520,8,22,7,1,0,29,4,22525029007011,1349
2,23755,8,16,5,1,2,31,11,16520031005010,1349
3,124139,8,16,5,1,2,21,8,16513021005006,1349
4,78671,8,15,5,0,3,20,14,15506020005008,1349
5,79085,8,13,4,1,3,19,8,13511019004004,1349
6,122169,8,9,2,3,3,12,12,9500012002005,1349
7,89569,8,9,3,0,5,11,15,9496011003007,1349
8,79866,8,8,2,2,4,17,10,8507017002009,1349
9,80293,8,4,1,1,6,5,16,4489005001003,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()